In [1]:
from pycorenlp import StanfordCoreNLP
import json

nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

def get_ner(sentence):
    result = dependency_parser(sentence, properties={
        "outputFormat": "json",
        "annotators": "ner"})
    res = []
    for obj in result['sentences'][0]['entitymentions']:
        res.append((obj['text'], obj['ner']))
    return res

In [2]:
get_ner("you owe obama $4 in indonesia")

[('$4', 'MONEY'), ('indonesia', 'COUNTRY')]

In [14]:
import pandas as pd
from tqdm import  tqdm_notebook

def run():
    df = pd.read_csv("dataset/res16_baru.csv")
    sf = pd.DataFrame(columns = ['id','review','target','category','term'])
    count = 0
    all_ner = []
    money_ner = []
    title_ner = []
    for index in tqdm_notebook(range(len(df))):
        new_term = []
        ner = get_ner(df['review'][index])
        if len(ner):
#             print(df['review'][index])
#             print(ner)
#             print()
            for word, tipe in ner:
                if tipe != 'TITLE' and tipe != 'MONEY':
                    all_ner.append(word)
                elif tipe == 'MONEY':
                    money_ner.append(word)
    return all_ner, money_ner, title_ner
#         for nn in get_nouns(df['review'][index]):
#             new_term.append(nn)
#         if len(new_term) == 0:
#             new_term = '|'.join(get_nouns(df['review'][index]))
#             count += 1
        
#         sf = sf.append({'id': df['id'][index], 
#                     'review': df['review'][index],
#                     'category': df['category'][index],
#                     'term': '|'.join(new_term)
#                    }, ignore_index=True)
#     print(count)

#     sf.to_csv("Results/add_noun/"+ tipe +".csv")
#     sf.to_excel("Results/add_noun/"+ tipe +".xlsx")

In [15]:
new_ner, mon_ner, tit_ner = run()

HBox(children=(IntProgress(value=0, max=464), HTML(value='')))

In [16]:
splitted_ner = []
for ner in new_ner:
    for word in ner.split(' '):
        splitted_ner.append(word.lower())
ner_ner = sorted(list(dict.fromkeys(splitted_ner)))

In [17]:
splitted_ner = []
for ner in mon_ner:
    for word in ner.split(' '):
        splitted_ner.append(word.lower())
new_mon = sorted(list(dict.fromkeys(splitted_ner)))

In [11]:
import pandas as pd

def run_filter(tipe, where, ner):
    df = pd.read_csv("Results/" + where + tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','target', 'category','term'])
    for index, concat_terms in enumerate(df['term']):
        new_terms = []
        try:
            terms = concat_terms.split('|')
        except:
            terms = []
        for term in terms:
            if term not in ner and len(term) > 3:
                new_terms.append(term)
        sentence = df['review'][index].lower()
        sf = sf.append({'id': df['id'][index], 
                        'review': df['review'][index],
                        'target': df['target'][index],
                        'category': df['category'][index],
                        'term': '|'.join(new_terms)
                       }, ignore_index=True)

    sf.to_csv("Results/Filter/"+ tipe +"-ner.csv")
    sf.to_excel("Results/Filter/"+ tipe +"-ner.xlsx")

In [10]:
run_filter("16_lda2vec-partial", 'add_noun/', ner_ner)

In [12]:
run_filter("16_lda2vec-op-partial", 'add_noun/', ner_ner)

In [29]:
import pandas as pd

def run_price(tipe, where):
    df = pd.read_csv("Results/" + where + tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','target', 'label','predict', 'terms'])
    for index, reviews in enumerate(df['review']):
        tokenized = reviews.lower().replace('  ', ' ').strip().split(' ')
        predict = df['predict'][index]
        for token in tokenized:
            for money in mon_ner:
                if money in token:
                    if predict == '':
                        predict += 'PRICES'
                    else:
                        predict += '|PRICES'
        sentence = df['review'][index].lower()
        sf = sf.append({'id': df['id'][index], 
                        'review': df['review'][index],
                        'target': df['target'][index],
                        'label': df['label'][index],
                        'predict': predict,
                        'terms': df['terms'][index]
                       }, ignore_index=True)

    sf.to_csv("Results/Categorization/"+ tipe +"-money.csv")
    sf.to_excel("Results/Categorization/"+ tipe +"-money.xlsx")

In [30]:
run_price('16_lda2vec-partial-ner', 'Categorization/')